In [80]:
import pandas as pd
import numpy as np
import utils

In [109]:
path = ""
#path = "../data/old_labels/"
#path "../data/new_labels/"
test = pd.read_csv(path + "test_subset.csv")
test["label"] = pd.to_numeric(test["label"], errors='coerce')
train_1 = pd.read_csv(path + "train_first_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_2 = pd.read_csv(path + "train_second_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
#train_3 = pd.read_csv(path + "train_third_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
extra_neg = pd.read_csv(path + "extra_neg.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
#train_1["label"] = pd.to_numeric(train_1["label"], errors='coerce')
#train = pd.concat([train_1,train_2,train_3],axis=0)
train = pd.concat([train_1,train_2,extra_neg],axis=0)
val = pd.read_csv(path + "val_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]

test = test.reset_index()
train=train.reset_index()
val = val.reset_index()

train.head()

,index,restaurant_code,review_title,review_body,account_name,account_id,aspect,phrase,label,new_aspect_1,new_aspect_2
0,0,2lhaZp7B,So glad the weekend is here and I can finally ...,Breakfast with the husband this morning at Gen...,Maureen Ow,@72128,food,"cheat, day",0.0,NaN,NaN
1,1,2lhaZp7B,So glad the weekend is here and I can finally ...,Breakfast with the husband this morning at Gen...,Maureen Ow,@72128,food,"glutinous, rice",0.0,food,NaN
2,2,2lhaZp7B,My Saturday morning started with breakfast at ...,It is so inspiring talking to Uncle Kun as he ...,Maureen Ow,@72128,food,"cantonese, style, congee, well, seasoned, gene...",1.0,food,food
3,3,2lhaZp7B,My Saturday morning started with breakfast at ...,It is so inspiring talking to Uncle Kun as he ...,Maureen Ow,@72128,portion,"well, seasoned, generous, ingredients",1.0,portion,portion
4,4,2lhaZp7B,03 Jan 2018 my comfort food..,Lunar New Year is approaching.. 根叔 Gen Shu is ...,Bobcatsysop YK Chan,@BobcatSysOp,time,"sold, around, lunch, time",0.0,NaN,NaN


In [ ]:
for data in [train_1,train_2,train_3,train,val,test]:
    print("Total size = {}".format(data.shape[0]))
    print("Noise = {} ".format(data.loc[(data.label == 0.0) & (data.new_aspect_1.isnull()) & (data.new_aspect_2.isnull())].shape[0]))
    print("Aspect with no adjectives = {} ".format(data.loc[(data.new_aspect_1.notnull()) & (data.new_aspect_2.isnull())].shape[0]))
    print("Aspect with adjectives = {} ".format(data.loc[(data.new_aspect_1.notnull()) & (data.new_aspect_2.notnull())].shape[0]))
    print("")

In [41]:
test.head()

,index,Unnamed: 0,restaurant_code,review_title,review_body,account_name,account_id,aspect,phrase,phrase_no_aspect,phrase_no_noun,label,new_aspect_1,new_aspect_2,phrase_lemma,phrase_stem
0,0,0,328-katong-laksa-united-square,328 Katong Laksa; Now you don't have to travel...,Back then during the 2013 hawker heroes challe...,Qing Xiang,@qingxiang,food,katong laksa travel way,"['katong', 'travel', 'way']","['katong', 'laksa', 'travel']",0,NaN,NaN,katong laksa travel way,katong laksa travel way
1,1,1,328-katong-laksa-united-square,328 Katong Laksa; Now you don't have to travel...,Back then during the 2013 hawker heroes challe...,Qing Xiang,@qingxiang,food,east centralised outlet united square shopping...,"['east', 'centralised', 'outlet', 'united', 's...","['centralised', 'united', 'square', 'shopping']",0,NaN,NaN,east centralised outlet united square shopping...,east centralis outlet unit squar shop mall novena
2,2,2,328-katong-laksa-united-square,328 Katong Laksa; Now you don't have to travel...,Back then during the 2013 hawker heroes challe...,Qing Xiang,@qingxiang,food,laksa defeated gordon ramsay version,"['defeated', 'gordon', 'ramsay', 'version']","['laksa', 'defeated', 'ramsay']",1,food,food,laksa defeated gordon ramsay version,laksa defeat gordon ramsay version
3,3,3,328-katong-laksa-united-square,328 Katong Laksa; Now you don't have to travel...,Back then during the 2013 hawker heroes challe...,Qing Xiang,@qingxiang,food,laksa category,['category'],['laksa'],0,NaN,NaN,laksa category,laksa categori
4,4,4,328-katong-laksa-united-square,328 Katong Laksa; Now you don't have to travel...,Back then during the 2013 hawker heroes challe...,Qing Xiang,@qingxiang,food,definitely live,"['definitely', 'live']","['definitely', 'live']",1,food,food,definitely live,definit live


####  Data Preprocessing

1. Clean Phrase (Utils) 
2. Filter the phrase column where there is an empty string 
3. Combine the aspects into 1 & aggregate the sentiment scores by averaging then sign(>0 = 1,=0 =0 <0 = -1)


In [110]:
# clean phrases
train["phrase_lemma"] = train.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=True, stem=False))
val["phrase_lemma"] = val.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=True, stem=False))
test["phrase_lemma"] = test.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=True, stem=False))

train["phrase_stem"] = train.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=False, stem=True))
val["phrase_stem"] = val.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=False, stem=True))
test["phrase_stem"] = test.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=False, stem=True))

train.phrase = train.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=False, stem=False))
val.phrase = val.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=False, stem=False))
test.phrase = test.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=False, stem=False))

In [111]:
# filter out rows where there are no characters
train = train.loc[(train.phrase.str.len() > 0)]
val = val.loc[(val.phrase.str.len() > 0)]
test = test.loc[(test.phrase.str.len() > 0)]

In [112]:
# if nan label, replace with 0
train.label = train.label.apply(lambda x: 0 if np.isnan(x) else x)
val.label = val.label.apply(lambda x: 0 if np.isnan(x) else x)
test.label = test.label.apply(lambda x: 0 if np.isnan(x) else x)

In [113]:
train.head()

,index,restaurant_code,review_title,review_body,account_name,account_id,aspect,phrase,label,new_aspect_1,new_aspect_2,phrase_lemma,phrase_stem
0,0,2lhaZp7B,So glad the weekend is here and I can finally ...,Breakfast with the husband this morning at Gen...,Maureen Ow,@72128,food,cheat day,0.0,NaN,NaN,cheat day,cheat day
1,1,2lhaZp7B,So glad the weekend is here and I can finally ...,Breakfast with the husband this morning at Gen...,Maureen Ow,@72128,food,glutinous rice,0.0,food,NaN,glutinous rice,glutin rice
2,2,2lhaZp7B,My Saturday morning started with breakfast at ...,It is so inspiring talking to Uncle Kun as he ...,Maureen Ow,@72128,food,cantonese style congee seasoned generous,1.0,food,food,cantonese style congee seasoned generous,cantones style conge season gener
3,3,2lhaZp7B,My Saturday morning started with breakfast at ...,It is so inspiring talking to Uncle Kun as he ...,Maureen Ow,@72128,portion,seasoned generous ingredients,1.0,portion,portion,seasoned generous ingredient,season gener ingredi
4,4,2lhaZp7B,03 Jan 2018 my comfort food..,Lunar New Year is approaching.. 根叔 Gen Shu is ...,Bobcatsysop YK Chan,@BobcatSysOp,time,sold lunch time,0.0,NaN,NaN,sold lunch time,sold lunch time


In [114]:
def sgn(x):
    if x < 0:
        return -1
    elif x == 0:
        return 0
    else:
        return 1

In [115]:
# filter out phrases with hashtags & those with character length <=5 then get the majority (might not make sense)
# train
trainagg = train.groupby(["restaurant_code","review_title","account_name","new_aspect_1"]).agg({"phrase": " ".join,
                                                                                                "phrase_lemma": " ".join,
                                                                                                "phrase_stem":" ".join,
                                                                                                "label": "mean"})
trainagg = trainagg.reset_index()
trainagg.label = trainagg.label.map(sgn)


# test
valagg = val.groupby(["restaurant_code","review_title","account_name","new_aspect_1"]).agg({"phrase": " ".join,
                                                                                                "phrase_lemma": " ".join,
                                                                                                "phrase_stem":" ".join,
                                                                                                "label": "mean"})
valagg = valagg.reset_index()
valagg.label = valagg.label.map(sgn)

#val 
testagg = test.groupby(["restaurant_code","review_title","account_name","new_aspect_1"]).agg({"phrase": " ".join,
                                                                                                "phrase_lemma": " ".join,
                                                                                                "phrase_stem":" ".join,
                                                                                                "label": "mean"})
testagg = testagg.reset_index()
testagg.label = testagg.label.map(sgn)


In [116]:
trainvalagg = pd.concat([trainagg, valagg],axis=0)

In [117]:
testagg.label.value_counts()

 1    372
 0    120
-1     40
Name: label, dtype: int64

In [66]:
#path = "../data/new_labels/"
trainagg.to_csv(path + "train_oldpreproc.csv")
valagg.to_csv(path + "val_oldpreproc.csv")
testagg.to_csv(path + "test_oldpreproc.csv")

In [67]:
testagg.shape

(383, 8)

###### Text Vectorisation --> used on data2
1. BoW
2. Tf-idf
3. Fasttext - can handle oov words https://www.quora.com/How-does-fastText-output-a-vector-for-a-word-that-is-not-in-the-pre-trained-model (not used)


In [118]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
import re
from tqdm import tqdm

In [119]:
bow = CountVectorizer(analyzer='word',lowercase=True,ngram_range=(1, 3))
bow_train = bow.fit_transform(trainvalagg.phrase)
bow_test = bow.transform(testagg.phrase)

In [120]:
tfidf = TfidfVectorizer(analyzer='word',lowercase=True,ngram_range=(1, 3))
tfidf_train = tfidf.fit_transform(trainvalagg.phrase)
tfidf_test = tfidf.transform(testagg.phrase)

In [ ]:
"""
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec

# Preprocess data for fasttext word embeddings training
train_clean = [process_text(sentence) for sentence in tqdm(trainvalagg.phrase) if sentence.strip() !='']
word_tokenizer = nltk.WordPunctTokenizer()
train_word_tokens = [word_tokenizer.tokenize(sent) for sent in tqdm(train_clean)]

# Preprocess data for fasttext word embeddings testing data
test_clean = [process_text(sentence) for sentence in tqdm(test2.phrase) if sentence.strip() !='']
word_tokenizer = nltk.WordPunctTokenizer()
test_word_tokens = [word_tokenizer.tokenize(sent) for sent in tqdm(test_clean)]

# Train Fasttext word embeddings on training data
# Defining values for parameters
embedding_size = 300
window_size = 5
min_word = 5
down_sampling = 1e-2
 

fast_Text_model = FastText(train_word_tokens,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers = 4,
                      sg=1,
                      iter=100)
"""

In [ ]:
"""
# Save fastText gensim model
fast_Text_model.save("ftmodel/ft_model_train")
# Load saved gensim fastText model
fast_Text_model = Word2Vec.load("ftmodel/ft_model_train") """

In [ ]:
"""
# convert sentence --> average(each word embedding)
fasttext_train = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in train_word_tokens]
# convert sentence --> average(each word embedding)
fasttext_test = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in test_word_tokens]
"""

In [ ]:
"""
fasttext_train_processed = []
for row in fasttext_train:
    try:
        fasttext_train_processed.append(list(row))
    except:
        fasttext_train_processed.append([0]*300)

fasttext_test_processed = []
for row in fasttext_test:
    try:
        fasttext_test_processed.append(list(row))
    except:
        fasttext_test_processed.append([0]*300)
"""

In [ ]:
"""
fasttext_train_processed = np.array(fasttext_train_processed).reshape(-1,300)
fasttext_test_processed = np.array(fasttext_test_processed).reshape(-1,300)
"""

##### Modelling

1. Naive Bayes/Support Vector Machines (NBSVM)/Logistic Regression

(Baseline) 1. Naive Bayes -Support Vector Machines (NBSVM)- Logistic Regression

In [121]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC

logreg = LogisticRegression()
nb = MultinomialNB()
svm = SVC()

all_train = [bow_train,tfidf_train]
all_test = [bow_test,tfidf_test]
all_vectorisers = ["BoW","TfIdf"]

for train,test,vectoriser in zip(all_train,all_test,all_vectorisers):
    logreg.fit(train,trainvalagg.label)
    
    print("Logistic Regression + " + vectoriser)
    print("Training Data")
    print(classification_report(trainvalagg.label, logreg.predict(train)))
    print("Test Data")
    print(classification_report(testagg.label, logreg.predict(test))) 
    
    svm.fit(train,trainvalagg.label)
    
    print("SVM + " + vectoriser)
    print("Training Data")
    print(classification_report(trainvalagg.label, svm.predict(train)))
    print("Test Data")
    print(classification_report(testagg.label, svm.predict(test))) 

    # Fasttest dont work w NaiveBayes
    nb.fit(train,trainvalagg.label)
    print("Naive Bayes + "+ vectoriser)
    print("Training Data")
    print(classification_report(trainvalagg.label, nb.predict(train)))
    print("Test Data")
    print(classification_report(testagg.label, nb.predict(test)))    

Logistic Regression + BoW
Training Data
              precision    recall  f1-score   support

          -1       0.99      0.96      0.98       397
           0       0.98      0.99      0.99      1015
           1       0.99      0.99      0.99      1318

    accuracy                           0.99      2730
   macro avg       0.99      0.98      0.99      2730
weighted avg       0.99      0.99      0.99      2730

Test Data
              precision    recall  f1-score   support

          -1       0.52      0.30      0.38        40
           0       0.43      0.82      0.56       120
           1       0.89      0.67      0.77       372

    accuracy                           0.68       532
   macro avg       0.61      0.60      0.57       532
weighted avg       0.76      0.68      0.69       532

SVM + BoW
Training Data
              precision    recall  f1-score   support

          -1       0.97      0.68      0.80       397
           0       0.79      0.97      0.87      1015
 